In [2]:
#author: ML Tlachac

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from textblob import TextBlob
import textblob as tb

from empath import Empath
lexicon = Empath()
lexicon.analyze("text", normalize=True)

In [13]:
#load data outputed from data preprocessing
pDF = pd.read_csv("TextSetupDFa.csv")

#add features not captured in feature engineering code
nTags = []
for i in range(0, len(pDF)):
    nTags.append(len(pDF.Tags[i].split(",")))
pDF["nTags"] = nTags
nChar = []
for i in range(0, len(pDF)):
    nChar.append(len(pDF.Content[i]))
pDF["nChar"] = nChar

#remove columns not used in model- command changed with version
#pDF = pDF.drop(columns = ['Unnamed: 0', 'Address', "Content", "posTags", "Polarity", "Subjectivity"])
pDF = pDF.drop(['Unnamed: 0', 'Address', "Content", "posTags", "Polarity", "Subjectivity", "Tags"], axis = 1)

#outputs the released dataset
pDF.to_csv("pDF.csv", index=False)

In [ ]:
#w is user given parameter a, r, and w in the paper
#values can be integers or decimals
w = 0.5

In [ ]:
#Combine rows into lists for each participant

colSet = list(set(pDF.columns))
setID = list(set(pDF.ID))

for c in colSet:
    tlist = []
    nlist = []
    slist = []
    for i in range(0, len(setID)):
        tempDF = pDF[pDF.ID == setID[i]]
        tempDF = tempDF.reset_index()
        templist = []
        for j in range(0, tempDF.shape[0]):
            templist.append(tempDF[c][j])
        if c == "ID":
            tlist.append(templist[0])
            nlist.append(tempDF.shape[0])
        else:
            tlist.append(templist)
        if c == "Texts":
            slist.append(sum(templist))
    idDF[c] = tlist
    if c == "ID":
        idDF["nAddress"] = nlist
    if c == "Texts":
        idDF["nTexts"] = slist


In [ ]:
#w option 1: can use if w is an integer

#k = []
#k_nAddress = []
#for i in range(0, len(setID)):
#    tempDF = pDF[pDF.ID == setID[i]]
#    tempDF = tempDF.set_index(np.arange(0,tempDF.shape[0]))
#    tempDF2 = tempDF.sort_values(by = "Texts", ascending = False)
#    list2 = list(tempDF2.index[0:w])
#    k.append(list2)
#    k_nAddress.append(len(list2))

In [ ]:
#w option 2: can use if w is a decimal

k = []
k_nAddress = []
for i in range(0, len(setID)):
    tempDF = pDF[pDF.ID == setID[i]]
    tempDF = tempDF.set_index(np.arange(0,tempDF.shape[0]))
    tempDF2 = tempDF.sort_values(by = "Texts", ascending = False)
    threshold = int(round(w*tempDF.shape[0], 0))
    if threshold == 0:
        list2 = []
        list2.append(tempDF2.index[0])
    else:
        list2 = list(tempDF2.index[0:threshold])
    k.append(list2)
    k_nAddress.append(len(list2))

In [ ]:
#get top 1 and top k
#w option 3: can use if w is a decimal - commented out

#k = []
one = []
n = []
#k_nAddress = []
#k_pct = []

for i in range(0, idDF.shape[0]):
    #setT = sorted(idDF.Texts[i])
    maxT = max(idDF.Texts[i])
    #kcontacts = []
    onecontacts = []
    n.append(len(idDF.Address[i]))
    for j in range(0, len(idDF.Address[i])):
        if idDF.Texts[i][j] == maxT:
            onecontacts.append(j)
        #if idDF.Texts[i][j] >= w*maxT:
            #kcontacts.append(j)
    one.append(onecontacts[0])
    #k.append(kcontacts)
    #k_nAddress.append(len(kcontacts))
    #k_pct.append(len(kcontacts)/idDF.nAddress[i])

idDF["k"] = k
idDF["t"] = one
idDF["k_nAddress"] = k_nAddress
idDF["nAddress"] = n
idDF[0:2]

In [ ]:
#read file with PHQ-9 scores and extract needed information
phqDF = pd.read_csv("phq.csv")
phqDF2 = pd.DataFrame()
phqDF2["ID"] = phqDF["ID"]
phqDF2["PHQ"] = phqDF["PHQ"]
phqDF2 = phqDF2.dropna()
phqDF2 = phqDF2.astype("int64")

#merge dataframes
idDF = idDF.merge(phqDF2, on = "ID")

In [ ]:
#new dataframes for n, k, and t patients

nDF = pd.DataFrame()
kDF = pd.DataFrame()
tDF = pd.DataFrame()
nDF["ID"] = idDF["ID"]
kDF["ID"] = idDF["ID"]
tDF["ID"] = idDF["ID"]
kDF["k_nAddress"] = idDF["k_nAddress"]
nDF["n_nAddress"] = idDF["nAddress"]
tDF["n_nAddress"] = idDF["nAddress"]

col = list(idDF.columns[:-4])

k_col = []
t_col = []
n_col = []
for c in col:
    k_col.append("k_" + c)
    t_col.append("t_" + c)
    n_col.append("n_" + c)
    
for c in range(1, len(col)):
    n_templist = []
    t_templist = []
    k_templist = []
    for i in range(0, idDF.shape[0]):
        t_templist.append(idDF[col[c]][i][idDF.t[i]])
        n_templist.append(sum(idDF[col[c]][i]))
        k_templist2 = []
        for j in range(0, len(idDF.k[i])):
            k_templist2.append(idDF[col[c]][i][idDF.k[i][j]])
        k_templist.append(sum(k_templist2))
    tDF[t_col[c]] = t_templist
    kDF[k_col[c]] = k_templist
    nDF[n_col[c]] = n_templist

nDF["PHQ"] = idDF["PHQ"]
kDF["PHQ"] = idDF["PHQ"]
tDF["PHQ"] = idDF["PHQ"]

In [ ]:
#change middle columns into averages or percents

n_col = list(nDF)
k_col = list(kDF)
t_col = list(tDF)
nDF2 = pd.DataFrame()
kDF2 = pd.DataFrame()
tDF2 = pd.DataFrame()
for i in range(0, len(n_col)):
    if (i < 3) | (i == nDF.shape[1]-1):
        nDF2[n_col[i]] = nDF[n_col[i]]
        kDF2[k_col[i]] = kDF[k_col[i]]
        tDF2[t_col[i]] = tDF[t_col[i]] 
    elif i < 8:
        n_avg = []
        k_avg = []
        t_avg = []
        for j in range(0, nDF.shape[0]):
            n_avg.append(nDF[n_col[i]][j]/nDF["n_Texts"][j])
            k_avg.append(kDF[k_col[i]][j]/kDF["k_Texts"][j])
            t_avg.append(tDF[t_col[i]][j]/tDF["t_Texts"][j])
        nDF2[n_col[i]] = n_avg
        kDF2[k_col[i]] = k_avg
        tDF2[t_col[i]] = t_avg
    elif ((i > 7) & (i < 11)):
        n_avg = []
        k_avg = []
        t_avg = []
        for j in range(0, nDF.shape[0]):
            n_avg.append(nDF[n_col[i]][j]/nDF["n_nAddress"][j])
            k_avg.append(kDF[k_col[i]][j]/kDF["k_nAddress"][j])
        nDF2[n_col[i]] = n_avg
        kDF2[k_col[i]] = k_avg
        tDF2[t_col[i]] = tDF[t_col[i]]    
    else:
        n_avg = []
        k_avg = []
        t_avg = []
        for j in range(0, nDF.shape[0]):
            n_avg.append(nDF[n_col[i]][j]/nDF["n_nTags"][j])
            k_avg.append(kDF[k_col[i]][j]/kDF["k_nTags"][j])
            t_avg.append(tDF[t_col[i]][j]/tDF["t_nTags"][j])
        nDF2[n_col[i]] = n_avg 
        kDF2[k_col[i]] = k_avg
        tDF2[t_col[i]] = t_avg
        
#save dataframes that will be fed into machine learning model
nDF2.to_csv("nAllTBdfsd.csv")
kDF2.to_csv("kTBdf" + str(w) + "pctsd.csv")
tDF2.to_csv("tAllTBdfsd.csv")